# Yelp Reviews Silver Layer Transformation

This notebook ingests Yelp review data from the Bronze layer, parses JSON records, validates them, deduplicates, cleans the text, and transforms them into a Silver layer format using **PySpark RDDs**.

In [ ]:
import json
import re
import time
from datetime import datetime, timezone

import findspark

findspark.init()

try:
    from pyspark.sql import SparkSession
    from pyspark.sql.types import IntegerType, StringType, StructField, StructType

    pyspark_available = True
except ImportError:
    print("PySpark not available. Install with: pip install pyspark")
    pyspark_available = False

# Initialize SparkSession and SparkContext
if pyspark_available:
    spark = (
        SparkSession.builder.appName("yelp_review_silver_transform")
        .master("local[*]")
        .getOrCreate()
    )
    sc = spark.sparkContext

    print("Spark session initialzed succesfully!")
    print(f"Spark version: {spark.version}")
    print(f"Spark UI available at: {sc.uiWebUrl}")
else:
    print("Skipping Spark tasks - Pyspark not available")

## Utility Functions

In [ ]:
def parse_json_safe(json_str: str) -> dict:
    """
    Safely parse a JSON string and add ingestion metadata.

    Args:
        json_str (str): The JSON string to parse.
    Returns:
        dict: A dictionary containing the parsed data and ingestion metadata,
              or error information if parsing fails.
    """
    try:
        data = json.loads(json_str)

        # Add ingestion metadata
        data["_ingestion_date"] = datetime.now(timezone.utc).strftime("%Y-%m-%d")
        data["_ingestion_timestamp"] = time.time()
        data["_source"] = "yelp_dataset"
        data["_status"] = "valid"

        return data

    except json.JSONDecodeError as e:

        return {
            "_raw_data": json_str,
            "_ingestion_timestamp": time.time(),
            "_source": "yelp_dataset",
            "_status": "parse_error",
            "_error_msg": str(e),
        }

In [ ]:
def is_review_valid(r: dict) -> bool:
    """
    Validate a Yelp review record.

    Args:
        r (dict): The review record to validate.
    Returns:
        bool: True if the record is valid, False otherwise.
    """
    required = [
        "review_id",
        "user_id",
        "business_id",
        "stars",
        "date",
        "text",
        "useful",
        "funny",
        "cool",
    ]
    if not all(k in r for k in required):
        return False
    if r["text"] is None or len(r["text"].strip()) == 0:
        return False
    try:
        stars = int(r["stars"])
        if stars < 1 or stars > 5:
            return False

        for field in ["useful", "funny", "cool"]:
            if int(r[field]) < 0:
                return False

        # Try both possible formats
        try:
            datetime.strptime(r["date"], "%Y-%m-%d")
        except ValueError:
            datetime.strptime(r["date"], "%Y-%m-%d %H:%M:%S")

    except Exception:
        return False

    return True

In [ ]:
def clean_text(t: str) -> str:
    """
    Clean review text by removing HTML tags and unwanted characters.
    Args:
        t (str): The review text to clean.
    Returns:
        str: The cleaned review text.
    """
    if not t:
        return ""
    t = t.lower()
    t = re.sub(r"<[^>]+>", " ", t)  # remove HTML
    t = re.sub(r"[^a-z0-9\s.,!?']", " ", t)  # keep only safe chars
    t = re.sub(r"\s+", " ", t).strip()
    return t

In [ ]:
def transform_review_to_silver(rec: dict) -> dict:
    """
    Transform a valid review record to Silver layer format.
    Args:
        rec (dict): The valid review record.
    Returns:
        dict: The transformed review record for Silver layer.
    """
    return {
        "review_id": rec["review_id"],
        "user_id": rec["user_id"],
        "business_id": rec["business_id"],
        "stars": int(rec["stars"]),
        "text_clean": clean_text(rec["text"]),
        "date": rec["date"],
        "useful": int(rec["useful"]),
        "funny": int(rec["funny"]),
        "cool": int(rec["cool"]),
        "ingest_date": datetime.now(timezone.utc).strftime("%Y-%m-%d"),
    }

## Quick Sanitty Check

In [ ]:
!wc -l /data/bronze/yelp/raw/2025-11-13/yelp_academic_dataset_review.json

In [ ]:
!head -n 3 /data/bronze/yelp/raw/2025-11-13/yelp_academic_dataset_review.json

## Load Bronze data as RDD

In [ ]:
raw_path = "file:///data/bronze/yelp/raw/2025-11-13/yelp_academic_dataset_review.json"
if pyspark_available:
    raw_reviews_rdd = sc.textFile(raw_path)
    parsed_reviews_rdd = raw_reviews_rdd.map(parse_json_safe)
    print("Parsed record count:", parsed_reviews_rdd.count())
    print("Parsed sample line:", parsed_reviews_rdd.take(1))

## Filter Valid Reviews and invalid reviews

In [ ]:
if pyspark_available:
    valid_reviews_rdd = parsed_reviews_rdd.filter(lambda d: d["_status"] == "valid")
    invalid_reviews_rdd = parsed_reviews_rdd.filter(
        lambda d: d["_status"] == "parse_error"
    )

    total_count = parsed_reviews_rdd.count()
    invalid_count = invalid_reviews_rdd.count()
    print(
        f"Malformed records: {invalid_count}/{total_count} ({invalid_count/total_count*100:.2f}%)"
    )
    print(f"Valid records: {valid_reviews_rdd.count()}")

## Deduplicate reviews by `review_id`

In [ ]:
if pyspark_available:
    deduped_reviews_rdd = (
        valid_reviews_rdd.map(lambda r: (r["review_id"], r))
        .reduceByKey(lambda a, b: a)
        .map(lambda kv: kv[1])
    )

    print("After deduplication:", deduped_reviews_rdd.count())

## Apply Silver Tranfromation

In [ ]:
if pyspark_available:
    silver_reviews_rdd = deduped_reviews_rdd.map(transform_review_to_silver)
    print("Silver records ready:", silver_reviews_rdd.count())
    print("Sample silver record:", silver_reviews_rdd.take(1))

## Convert to a DataFrame (to be saved as Parquet later)

In [ ]:
if pyspark_available:
    silver_review_schema = StructType(
        [
            StructField("review_id", StringType(), nullable=False),
            StructField("user_id", StringType(), nullable=False),
            StructField("business_id", StringType(), nullable=False),
            StructField("stars", IntegerType(), nullable=False),
            StructField("text_clean", StringType(), nullable=False),
            StructField("date", StringType(), nullable=False),  # ISO 8601 string
            StructField("useful", IntegerType(), nullable=False),
            StructField("funny", IntegerType(), nullable=False),
            StructField("cool", IntegerType(), nullable=False),
            StructField("ingest_date", StringType(), nullable=False),
        ]
    )

    df_silver = spark.createDataFrame(silver_reviews_rdd, schema=silver_review_schema)
    df_silver.printSchema()
    df_silver.show(5, truncate=100)

## Write Silver data as Parquet

In [ ]:
if pyspark_available:
    # Path /data/silver/<source>/<table>/ingest_date=<date>/
    #      /data/silver/yelp/review/ingest_date=2025-11-13/
    silver_path = "/data/silver/yelp/review/"
    df_silver.write.mode("overwrite").partitionBy("ingest_date").parquet(silver_path)
    print(f"Silver layer written to {silver_path}")

## Cleanup

In [ ]:
if pyspark_available:
    spark.stop()
    print("Spark session stopped.")